- relational data model
- connect to athena and query data
- etl job in python
- save results to S3
- glue deployment
- build tables on redshift
- copy data to reshift

In [1]:
import boto3
import pandas as pd
from io import StringIO
import time

In [3]:
athena_client = boto3.client(
    "athena",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION,
)

In [4]:

query_response = athena_client.start_query_execution(
    QueryString="SELECT * FROM enigma_jhu_enigma_jhu",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        #"EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
while True:
    try:
        # This function only loads the first 1000 rows
        athena_client.get_query_results(
            QueryExecutionId=query_response["QueryExecutionId"]
        )
        break
    except Exception as err:
        if "not yet finished" in str(err):
            time.sleep(0.001)
        else:
            raise err

In [5]:

temp_file_location: str = "enigma_jhu.csv"
s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION,
)
s3_client.download_file(
    S3_BUCKET_NAME,
    f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
    temp_file_location,
)
enigma_jhu = pd.read_csv(temp_file_location)

In [6]:
enigma_jhu.head()

,fips,admin2,province_state,country_region,last_update,latitude,longitude,confirmed,deaths,recovered,active,combined_key,partition_0
0,NaN,NaN,Anhui,China,2020-01-22T17:00:00,31.826,117.226,1.0,NaN,NaN,NaN,"""Anhui",csv
1,NaN,NaN,Beijing,China,2020-01-22T17:00:00,40.182,116.414,14.0,NaN,NaN,NaN,"""Beijing",csv
2,NaN,NaN,Chongqing,China,2020-01-22T17:00:00,30.057,107.874,6.0,NaN,NaN,NaN,"""Chongqing",csv
3,NaN,NaN,Fujian,China,2020-01-22T17:00:00,26.079,117.987,1.0,NaN,NaN,NaN,"""Fujian",csv
4,NaN,NaN,Gansu,China,2020-01-22T17:00:00,36.061,103.834,NaN,NaN,NaN,NaN,"""Gansu",csv


In [7]:
query_response = athena_client.start_query_execution(
    QueryString="SELECT * FROM us_county",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        #"EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
# while True:
#     try:
#         # This function only loads the first 1000 rows
#         athena_client.get_query_results(
#             QueryExecutionId=query_response["QueryExecutionId"]
#         )
#         break
#     except Exception as err:
#         if "not yet finished" in str(err):
#             time.sleep(0.001)
#         else:
#             raise err

In [8]:
temp_file_location: str = "us_county.csv"
s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION,
)
s3_client.download_file(
    S3_BUCKET_NAME,
    f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
    temp_file_location,
)
us_country = pd.read_csv(temp_file_location)

In [9]:
us_country.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [10]:
query_response = athena_client.start_query_execution(
    QueryString="SELECT * FROM us_states",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        #"EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

In [11]:
temp_file_location: str = "us_states.csv"
s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION,
)
s3_client.download_file(
    S3_BUCKET_NAME,
    f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
    temp_file_location,
)
us_states = pd.read_csv(temp_file_location)

In [12]:
us_states.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [13]:
query_response = athena_client.start_query_execution(
    QueryString="SELECT * FROM countypopulation",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR
    },
)

In [14]:
temp_file_location: str = "countrypopulation.csv"
s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION,
)
s3_client.download_file(
    S3_BUCKET_NAME,
    f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
    temp_file_location,
)
countrypopulation = pd.read_csv(temp_file_location)

In [15]:
countrypopulation.head()

,id,id2,county,state,population estimate 2018
0,0500000US01001,1001,Autauga,Alabama,55601
1,0500000US01003,1003,Baldwin,Alabama,218022
2,0500000US01005,1005,Barbour,Alabama,24881
3,0500000US01007,1007,Bibb,Alabama,22400
4,0500000US01009,1009,Blount,Alabama,57840


In [16]:
query_response = athena_client.start_query_execution(
    QueryString="SELECT * FROM us_daily",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR
    },
)

In [17]:
temp_file_location: str = "us_daily.csv"
s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION,
)
s3_client.download_file(
    S3_BUCKET_NAME,
    f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
    temp_file_location,
)
us_daily = pd.read_csv(temp_file_location)

In [18]:
us_daily.head()

,date,states,positive,negative,pending,hospitalizedcurrently,hospitalizedcumulative,inicucurrently,inicucumulative,onventilatorcurrently,...,lastmodified,recovered,total,posneg,deathincrease,hospitalizedincrease,negativeincrease,positiveincrease,totaltestresultsincrease,hash
0,20210307,56,28755524.0,74579770.0,11808.0,40212.0,878613.0,8137.0,45475.0,2801.0,...,2021-03-07T24:00:00Z,NaN,0,0,839,726,130414,41265,1156241,8b26839690cd05c0cef69cb9ed85641a76b5e78e
1,20210306,56,28714259.0,74449356.0,11783.0,41401.0,877887.0,8409.0,45453.0,2811.0,...,2021-03-06T24:00:00Z,NaN,0,0,1674,503,142201,59620,1409138,d0c0482ea549c9d5c04a7c86acb6fc6a8095a592
2,20210305,56,28654639.0,74307155.0,12213.0,42541.0,877384.0,8634.0,45373.0,2889.0,...,2021-03-05T24:00:00Z,NaN,0,0,2221,2781,271917,68787,1744417,a35ea4289cec4bb55c9f29ae04ec0fd5ac4e0222
3,20210304,56,28585852.0,74035238.0,12405.0,44172.0,874603.0,8970.0,45293.0,2973.0,...,2021-03-04T24:00:00Z,NaN,0,0,1743,1530,177957,65487,1590984,a19ad6379a653834cbda3093791ad2c3b9fab5ff
4,20210303,56,28520365.0,73857281.0,11778.0,45462.0,873073.0,9359.0,45214.0,3094.0,...,2021-03-03T24:00:00Z,NaN,0,0,2449,2172,267001,66836,1406795,9e1d2afda1b0ec243060d6f68a7134d011c0cb2a


In [19]:
query_response = athena_client.start_query_execution(
    QueryString="SELECT * FROM hospital_beds_rearc_usa_hospital_beds",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
    },
)

In [20]:
temp_file_location: str = "hospital_beds.csv"
s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION,
)
s3_client.download_file(
    S3_BUCKET_NAME,
    f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
    temp_file_location,
)
hospital_beds = pd.read_csv(temp_file_location)

In [21]:
hospital_beds.head()

,objectid,hospital_name,hospital_type,hq_address,hq_address1,hq_city,hq_state,hq_zip_code,county_name,state_name,...,num_staffed_beds,num_icu_beds,adult_icu_beds,pedi_icu_beds,bed_utilization,avg_ventilator_usage,potential_increase_in_bed_capac,latitude,longtitude,partition_0
0,1,Phoenix VA Health Care System (AKA Carl T Hayd...,VA Hospital,650 E Indian School Rd,NaN,Phoenix,AZ,85012,Maricopa,Arizona,...,129.0,0,0,NaN,NaN,0.0,0,33.495498,-112.066157,json
1,2,Southern Arizona VA Health Care System,VA Hospital,3601 S 6th Ave,NaN,Tucson,AZ,85723,Pima,Arizona,...,295.0,2,2,NaN,NaN,2.0,0,32.181263,-110.965885,json
2,3,VA Central California Health Care System,VA Hospital,2615 E Clinton Ave,NaN,Fresno,CA,93703,Fresno,California,...,57.0,2,2,NaN,NaN,2.0,0,36.773324,-119.779742,json
3,4,VA Connecticut Healthcare System - West Haven ...,VA Hospital,950 Campbell Ave,NaN,West Haven,CT,6516,New Haven,Connecticut,...,216.0,1,1,NaN,NaN,2.0,0,41.284400,-72.957610,json
4,5,Wilmington VA Medical Center,VA Hospital,1601 Kirkwood Hwy,NaN,Wilmington,DE,19805,New Castle,Delaware,...,60.0,0,0,NaN,NaN,1.0,0,39.740206,-75.606532,json


In [22]:
query_response = athena_client.start_query_execution(
    QueryString="SELECT * FROM us_total_latest",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR
    },
)

In [23]:
temp_file_location: str = "us_total_latest.csv"
s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION,
)
s3_client.download_file(
    S3_BUCKET_NAME,
    f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
    temp_file_location,
)
us_total_latest = pd.read_csv(temp_file_location)

In [24]:
us_total_latest.head()

,positive,negative,pending,hospitalizedcurrently,hospitalizedcumulative,inicucurrently,inicucumulative,onventilatorcurrently,onventilatorcumulative,recovered,hash,lastmodified,death,hospitalized,total,totaltestresults,posneg,notes
0,1061101,5170081,2775,53793,111955,9486,4192,4712,373,153947,95064ba29ccbc20dbec397033dfe4b1f45137c99,2020-05-01T09:12:31.891Z,57266,111955,6233957,6231182,6231182,"""NOTE: """"total"""""


In [25]:
query_response = athena_client.start_query_execution(
    QueryString="SELECT * FROM states_daily",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR
    },
)

In [26]:
temp_file_location: str = "states_daily.csv"
s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION,
)
s3_client.download_file(
    S3_BUCKET_NAME,
    f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
    temp_file_location,
)
states_daily = pd.read_csv(temp_file_location)

In [27]:
states_daily.head()

,date,state,positive,probablecases,negative,pending,totaltestresultssource,totaltestresults,hospitalizedcurrently,hospitalizedcumulative,...,dataqualitygrade,deathincrease,hospitalizedincrease,hash,commercialscore,negativeregularscore,negativescore,positivescore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [28]:
query_response = athena_client.start_query_execution(
    QueryString="SELECT * FROM state_abv",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR
    },
)

In [29]:
temp_file_location: str = "state_abv.csv"
s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION,
)
s3_client.download_file(
    S3_BUCKET_NAME,
    f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
    temp_file_location,
)
state_abv = pd.read_csv(temp_file_location)

In [30]:
state_abv.head() 
# noted data quality issue, column headers shift a step further below

,col0,col1
0,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR


In [31]:
query_response = athena_client.start_query_execution(
    QueryString="SELECT * FROM countrycode",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR
    },
)

In [32]:
temp_file_location: str = "countrycode.csv"
s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION,
)
s3_client.download_file(
    S3_BUCKET_NAME,
    f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
    temp_file_location,
)
countrycode = pd.read_csv(temp_file_location)

In [33]:
countrycode.head()

,country,alpha-2 code,alpha-3 code,numeric code,latitude,longitude
0,Afghanistan,AF,AFG,4.0,33.0000,65.0
1,Albania,AL,ALB,8.0,41.0000,20.0
2,Algeria,DZ,DZA,12.0,28.0000,3.0
3,American Samoa,AS,ASM,16.0,-14.3333,-170.0
4,Andorra,AD,AND,20.0,42.5000,1.6


# data transformation

In [34]:
new_header = state_abv.iloc[0] # get the first row for the dataset
state_abv = state_abv[1:] # take out the data  header
state_abv.columns = new_header # set the hearder row as the df header

In [35]:
state_abv.head()

,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR
5,California,CA


# ETL job on athena and query data

In [36]:
factCovid_1 = enigma_jhu[['fips', 'province_state', 'country_region', 'confirmed', 'deaths', 'recovered', 'active']]
factCovid_2 = states_daily[['fips', 'date', 'positive', 'negative', 'hospitalizedcurrently', 'hospitalized', 'hospitalizeddischarged']]
factCovid = pd.merge(factCovid_1, factCovid_2, on='fips', how='inner')

In [37]:
factCovid.head()

,fips,province_state,country_region,confirmed,deaths,recovered,active,date,positive,negative,hospitalizedcurrently,hospitalized,hospitalizeddischarged
0,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210307,101327.0,305972.0,147.0,NaN,NaN
1,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210306,101327.0,305972.0,147.0,NaN,NaN
2,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210305,101066.0,305972.0,136.0,NaN,NaN
3,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210304,100867.0,305972.0,171.0,NaN,NaN
4,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210303,100765.0,305972.0,169.0,NaN,NaN


In [38]:
factCovid.shape

(26418, 13)

In [39]:
dimRegion_1 = enigma_jhu[['fips', 'province_state', 'country_region', 'latitude', 'longitude']]
dimRegion_2 = us_country[['fips', 'county', 'state']]
dimRegion = pd.merge(dimRegion_1, dimRegion_2, on='fips', how='inner')

In [40]:
dimRegion.head()

,fips,province_state,country_region,latitude,longitude,county,state
0,NaN,Anhui,China,31.826,117.226,New York City,New York
1,NaN,Anhui,China,31.826,117.226,Unknown,Rhode Island
2,NaN,Anhui,China,31.826,117.226,New York City,New York
3,NaN,Anhui,China,31.826,117.226,Unknown,Rhode Island
4,NaN,Anhui,China,31.826,117.226,New York City,New York


In [41]:
dimHospital = hospital_beds[['fips','state_name', 'latitude', 'longtitude', 'hq_address', 'hospital_name', 'hospital_type', 'hq_city', 'hq_state']]

In [42]:
dimHospital.head()

,fips,state_name,latitude,longtitude,hq_address,hospital_name,hospital_type,hq_city,hq_state
0,4013.0,Arizona,33.495498,-112.066157,650 E Indian School Rd,Phoenix VA Health Care System (AKA Carl T Hayd...,VA Hospital,Phoenix,AZ
1,4019.0,Arizona,32.181263,-110.965885,3601 S 6th Ave,Southern Arizona VA Health Care System,VA Hospital,Tucson,AZ
2,6019.0,California,36.773324,-119.779742,2615 E Clinton Ave,VA Central California Health Care System,VA Hospital,Fresno,CA
3,9009.0,Connecticut,41.284400,-72.957610,950 Campbell Ave,VA Connecticut Healthcare System - West Haven ...,VA Hospital,West Haven,CT
4,10003.0,Delaware,39.740206,-75.606532,1601 Kirkwood Hwy,Wilmington VA Medical Center,VA Hospital,Wilmington,DE


In [47]:
dimDate = states_daily[['fips', 'date']]

In [48]:
dimDate.head()

,fips,date
0,2,20210307
1,1,20210307
2,5,20210307
3,60,20210307
4,4,20210307


In [49]:
dimDate['date'] = pd.to_datetime(dimDate['date'], format='%Y%m%d') # change date type to datetime and change format as well

C:\Users\agyei\AppData\Local\Temp\ipykernel_11148\649841485.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['date'] = pd.to_datetime(dimDate['date'], format='%Y%m%d') # change date type to datetime and change format as well


In [50]:
dimDate['year'] = dimDate['date'].dt.year # extract year from the date into a new column
dimDate['month'] = dimDate['date'].dt.month # extract month from the date into a new column
dimDate['day_of_week'] = dimDate['date'].dt.dayofweek # extract week day from the date into a new column

C:\Users\agyei\AppData\Local\Temp\ipykernel_11148\1721187186.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['year'] = dimDate['date'].dt.year # extract year from the date into a new column
C:\Users\agyei\AppData\Local\Temp\ipykernel_11148\1721187186.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['month'] = dimDate['date'].dt.month # extract month from the date into a new column
C:\Users\agyei\AppData\Local\Temp\ipykernel_11148\1721187186.py:3: SettingWithCopyWarning: 
A value is tr

In [51]:
dimDate.head()

,fips,date,year,month,day_of_week
0,2,2021-03-07,2021,3,6
1,1,2021-03-07,2021,3,6
2,5,2021-03-07,2021,3,6
3,60,2021-03-07,2021,3,6
4,4,2021-03-07,2021,3,6


# Save aggregated data to s3

In [56]:
bucket = 'frank-covid-19-bucket'  #project bucket on S3

In [53]:
csv_buffer = StringIO()
csv_buffer

In [54]:
factCovid.to_csv(csv_buffer)

In [57]:
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'output/factCovid.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'GKW47M8YFXC76MC2',
  'HostId': 'oX/ttBV5YdYCf/vPeFOSgvtWr3M18aRei8wQk+FWV6/iGJjWXNs6RqwVeqFiO9PBOWcA3yvQlUE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'oX/ttBV5YdYCf/vPeFOSgvtWr3M18aRei8wQk+FWV6/iGJjWXNs6RqwVeqFiO9PBOWcA3yvQlUE=',
   'x-amz-request-id': 'GKW47M8YFXC76MC2',
   'date': 'Sat, 08 Jun 2024 15:21:36 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"6372f86ce96858dcb69570e9500bc3c7"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 2},
 'ETag': '"6372f86ce96858dcb69570e9500bc3c7"',
 'ServerSideEncryption': 'AES256'}

In [58]:
csv_buffer.getvalue()

',fips,province_state,country_region,confirmed,deaths,recovered,active,date,positive,negative,hospitalizedcurrently,hospitalized,hospitalizeddischarged\r\n0,72.0,Puerto Rico,US,3.0,0.0,0.0,,20210307,101327.0,305972.0,147.0,,\r\n1,72.0,Puerto Rico,US,3.0,0.0,0.0,,20210306,101327.0,305972.0,147.0,,\r\n2,72.0,Puerto Rico,US,3.0,0.0,0.0,,20210305,101066.0,305972.0,136.0,,\r\n3,72.0,Puerto Rico,US,3.0,0.0,0.0,,20210304,100867.0,305972.0,171.0,,\r\n4,72.0,Puerto Rico,US,3.0,0.0,0.0,,20210303,100765.0,305972.0,169.0,,\r\n5,72.0,Puerto Rico,US,3.0,0.0,0.0,,20210302,100735.0,305972.0,162.0,,\r\n6,72.0,Puerto Rico,US,3.0,0.0,0.0,,20210301,100584.0,305972.0,155.0,,\r\n7,72.0,Puerto Rico,US,3.0,0.0,0.0,,20210228,100297.0,305972.0,150.0,,\r\n8,72.0,Puerto Rico,US,3.0,0.0,0.0,,20210227,100044.0,305972.0,174.0,,\r\n9,72.0,Puerto Rico,US,3.0,0.0,0.0,,20210226,99860.0,305972.0,169.0,,\r\n10,72.0,Puerto Rico,US,3.0,0.0,0.0,,20210225,99619.0,305972.0,184.0,,\r\n11,72.0,Puerto Rico,US,3.0,0.0,0.0,,2021022

In [59]:
dimDate.to_csv(csv_buffer)

In [60]:
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'output/dimDate.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '66DPXBG28VR2C87W',
  'HostId': 'NZPJ8jTzk0VzO+3qXj2cgRWrBtmW2mcaZBZUG2h03vhaaTX72630QLgOufR1t4lVZwgwG2b+nW8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'NZPJ8jTzk0VzO+3qXj2cgRWrBtmW2mcaZBZUG2h03vhaaTX72630QLgOufR1t4lVZwgwG2b+nW8=',
   'x-amz-request-id': '66DPXBG28VR2C87W',
   'date': 'Sat, 08 Jun 2024 15:26:18 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"2526a7f8f9a076c05a84401d9f578077"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"2526a7f8f9a076c05a84401d9f578077"',
 'ServerSideEncryption': 'AES256'}

In [61]:
dimHospital.to_csv(csv_buffer)

In [62]:
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'output/Hospital.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'NRW1PCAA207NYY78',
  'HostId': '5nqhRCzUMoGPGnxhM48Z6IWI1rgzviL9s3E6HFdqBj2lDzyI1/klTGLnpBMvt86Q4gHu/yEXfeI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '5nqhRCzUMoGPGnxhM48Z6IWI1rgzviL9s3E6HFdqBj2lDzyI1/klTGLnpBMvt86Q4gHu/yEXfeI=',
   'x-amz-request-id': 'NRW1PCAA207NYY78',
   'date': 'Sat, 08 Jun 2024 15:27:49 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"bf6387963ea68e5ebba6ec73524c5c09"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"bf6387963ea68e5ebba6ec73524c5c09"',
 'ServerSideEncryption': 'AES256'}

In [71]:
dimRegion.to_csv(csv_buffer)

MemoryError: 

In [65]:
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'output/dimRegion.csv').put(Body=csv_buffer.getvalue())

MemoryError: 

In [69]:
import smart_open

s3_client = boto3.client('s3')

# Assuming 'large_file.csv' is a very large CSV
chunk_size = 10**6  # Adjust chunk size as necessary
chunks = pd.read_csv(dimHospital, chunksize=chunk_size)

with smart_open.open(f's3://{bucket}/output/dimRegion.csv', 'w', transport_params={'client': s3_client}) as fout:
    for i, chunk in enumerate(chunks):
        chunk.to_csv(fout, header=(i == 0), index=False)

TypeError: argument of type 'method' is not iterable